In [ ]:
import pandas as pd
import openai  # Example, you can replace with other APIs
import random
from typing import List, Dict, Callable
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

### This notebook was mainly for testing ###
Note that this gives some insight into the low level details of the workflow in evaluation.ipynb

In [2]:
df = pd.read_parquet("../data/066_IBM_HR/sample.parquet")
print(df.columns)

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')


In [3]:
summary = df.describe(include="all").to_string()
print(f"Data Overview:\n{summary}")

Data Overview:
            Age Attrition BusinessTravel    DailyRate              Department  DistanceFromHome  Education EducationField  EmployeeCount  EmployeeNumber  EnvironmentSatisfaction Gender  HourlyRate  JobInvolvement   JobLevel                JobRole  JobSatisfaction MaritalStatus  MonthlyIncome   MonthlyRate  NumCompaniesWorked Over18 OverTime  PercentSalaryHike  PerformanceRating  RelationshipSatisfaction  StandardHours  StockOptionLevel  TotalWorkingYears  TrainingTimesLastYear  WorkLifeBalance  YearsAtCompany  YearsInCurrentRole  YearsSinceLastPromotion  YearsWithCurrManager
count   20.0000        20             20    20.000000                      20         20.000000  20.000000             20           20.0       20.000000                20.000000     20   20.000000       20.000000  20.000000                     20        20.000000            20       20.00000     20.000000           20.000000     20       20          20.000000          20.000000                 20.000

In [ ]:
ZERO_SHOT = [
    "What is the average MonthlyIncome of employees in the Sales department?",
    "How many employees have a JobSatisfaction rating of 4?",
    "What is the most common JobRole among employees with high PerformanceRating?",
    "What percentage of employees have worked at more than 3 companies?",
    "Is there a correlation between YearsAtCompany and Attrition?",
]

In [5]:
FEW_SHOT = [
    {"question": "What is the average age of employees?", "answer": "The average age of employees is 35.65 years."},
    {"question": "How many employees work in the IT department?", "answer": "There are 50 employees in the IT department."},
    {"question": "What is the median MonthlyIncome?", "answer": "The median MonthlyIncome is $4629."},
]

Going to test different prompting methods:
- single shot
- few shot
- rag (generate context from data, using few shot and single shot)

In [6]:
def literal_context(df: pd.DataFrame) -> str:
    """Converts the entire DataFrame to a structured text format."""
    return f"Dataset:\n{df.to_string(index=False)}"

In [7]:
def summary_context(df: pd.DataFrame) -> str:
    summary = df.describe(include="all").to_string()
    return f"Data Overview:\n{summary}"

In [ ]:
def similarity_context(query: str, table: pd.DataFrame, embedder, k: int = 3) -> str:
    """
    Retrieves the top-k rows from a DataFrame that are most relevant to the query.
    
    Parameters:
        query (str): The user's query or question.
        table (pd.DataFrame): The tabular data containing information.
        embedder: An embedding model with an `encode` method that converts text into a vector.
        k (int): Number of top rows to retrieve (default is 3).
    
    Returns:
        str: A context string containing the retrieved rows, formatted for use in an LLM prompt.
    """
    # Convert each row of the table into a string (you can adjust the join separator as needed)
    row_texts = table.astype(str).agg(' | '.join, axis=1).tolist()
    # Compute the embedding for the query
    query_embedding = embedder.encode(query)
    row_embeddings = np.array([embedder.encode(text) for text in row_texts])
    similarities = cosine_similarity([query_embedding], row_embeddings)[0]
    # Identify the indices of the top-k most similar rows
    top_indices = np.argsort(similarities)[::-1][:k]
    retrieved_context = "\n".join([row_texts[i] for i in top_indices])    
    context = f"Retrieved relevant data:\n{retrieved_context}\n"
    return context


In [ ]:
def query_llm_zero_shot(model: str, question: str, context_method: Callable, data: pd.DataFrame) -> str:
    context = context_method(data)
    response = openai.ChatCompletion.create(
        model = model,
        messages = [
            {"role": "system", "content": "You are a data analyst answering questions about a company dataset."},
            {"role": "user", "content": f"Dataset Summary: {context}\n\nQuestion: {question}"},
        ],
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
def query_llm_few_shot(model: str, question: str, context_method: Callable, data: pd.DataFrame) -> str:
    """Sends a query to the specified LLM with few-shot examples."""
    context = context_method(data)
    messages = [
        {"role": "system", "content": "You are a data analyst answering questions about a company dataset."},
    ]
    
    # Add few-shot examples to improve response quality
    for example in FEW_SHOT:
        messages.append({"role": "user", "content": example["question"]})
        messages.append({"role": "assistant", "content": example["answer"]})
    
    # Add actual question and context
    messages.append({"role": "user", "content": f"Dataset Summary: {context}\n\nQuestion: {question}"})
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
    )
    return response["choices"][0]["message"]["content"]